# Selfdocprint

Provides enhanced and configurable `print()` functionality that allows for easily printing values such that they are: 

* *self-documented*
* *formatted*
* *nicely laid out*

Supports the keyword arguments: `file=` and `flush=`.

In [1]:
from selfdocprint import print

# some data
formula = "Ultimate Question of Life,\nthe Universe, and Everything"
theta = 6.006
x = 7


print(formula, theta, x, x*theta)


formula: Ultimate Question of Life,    theta: 6.006  x: 7  x * theta: 42.042
         the Universe, and Everything                                       


The default value for the `layout=` keyword argument (normally `None`) can be configured so that subsequent calls require even less typing. In this case it would be good practice to not call the function 'print' to avoid confusion with the expected behaviour of python's built-in `print()`.

In [7]:
prinline = PrintFunc(default_layout=InlineLayout())

prinline(formula, theta, x, x*theta)


formula: Ultimate Question of Life,    theta: 6.006  x: 7  x * theta: 42.042
         the Universe, and Everything                                       


A layout can be customised by overriding one or more of its fields (The fields are described in detail in the section: 'Style, format and layout specification'). Note that in the example below we override `head=` to suppress the empty line that is normally printed in front of the output when using the InlineLayout.

In [8]:
for i in range(0, 99, 21):
    print(i, i * theta, layout=InlineLayout(float_format="12.6f", pointer=" -> ", head=""))

i -> 0  i * theta ->     0.000000
i -> 21  i * theta ->   126.126000
i -> 42  i * theta ->   252.252000
i -> 63  i * theta ->   378.378000
i -> 84  i * theta ->   504.504000


Custom layouts can be created and referenced in subsequent calls. Note that in the example below we change the `style=` for the labels, which is specified as [ANSI Select Graphic Rendition](https://en.wikipedia.org/wiki/ANSI_escape_code#SGR_(Select_Graphic_Rendition)_parameters) parameters.

In [9]:
my_layout = InlineLayout(style="92;3", int_format="4")

print(x, x * theta, layout=my_layout)


x:    7  x * theta: 42.042


## Built-in layouts
Next to the *'inline'* layout three additional layouts are available. The example below demonstrates the *'dict'* layout. In addition it shows the use of the `beg=` keyword argument which can be used to provide a heading to the output.

In [10]:
from selfdocprint import DictLayout

print(formula, theta, x, x*theta, layout=DictLayout(), beg="Using the 'dict' layout:\n")

Using the 'dict' layout:

formula   : Ultimate Question of Life,  
            the Universe, and Everything
theta     :        6.006
x         :        7
x * theta :       42.042


For values that take up a lot of horizontal space the *'scroll'* layout is particularly useful.

In [11]:
from selfdocprint import ScrollLayout
import numpy as np

X = np.random.rand(5,5)

print(X, X.T @ X, layout=ScrollLayout(), beg="Using the 'scroll' layout:\n")

Using the 'scroll' layout:

X:
[[0.97621221 0.16093693 0.61981285 0.95346398 0.93562747]
 [0.90888155 0.01872998 0.06674502 0.9526218  0.21333503]
 [0.67089376 0.33947049 0.93339294 0.63176867 0.5430394 ]
 [0.57586679 0.61559037 0.18789235 0.65569875 0.27490041]
 [0.71899852 0.63343481 0.53428263 0.73443678 0.98041046]]

X.T @ X:
[[3.07773582 1.2118173  1.78428908 3.1261073  2.33480867]
 [1.2118173  0.92168287 0.87195821 1.25461662 1.12917078]
 [1.78428908 0.87195821 1.58060671 1.759838   1.67649009]
 [3.1261073  1.25461662 1.759838   3.18505173 2.33869133]
 [2.33480867 1.12917078 1.67649009 2.33869133 2.2525773 ]]



Use the *'minimal'* layout if you only want to print labels in front of the values.

In [12]:
from selfdocprint import MinimalLayout

print(formula, theta, x, theta*x, layout=MinimalLayout(), beg="Using the 'minimal' layout:\n\n")

Using the 'minimal' layout:

formula:Ultimate Question of Life,
the Universe, and Everything theta:6.006 x:7 theta * x:42.042


## Style, format and layout specification
The `selfdocprint.print_layout_specs()` function prints the specification for every built-in layout and a rudimentary description of the layout algorithm.

In [13]:
import selfdocprint as sdp

sdp.print_layout_specs()


                   layout :  Inline     Dict       Scroll     Minimal
lbl_format                :  ''         '<'        ''         ''         # applied to labels
int_format                :  ''         '-8'       ''         ''         # applied to values of type int
float_format              :  ''         '-12.3f'   ''         ''         # applied to values of type float
str_format                :  '<'        '<'        ''         ''         # applied to all other value types after conversion to str
head                      :  '\n'       '\n'       '\n'       ''         # printed at the beginning of the output
seperator.join([          :  '  '       '\n'       '\n\n'     ' '        # printed inbetween the styled and formatted label/value pairs
    style(<label> +       :  '95'       '95'       '95'       '95'       # style applied to the concatenation of <label> and pointer
    pointer) + <value>    :  ': '       ' : '      ':\n'      ':'        # printed in between the <label> and

All parameters are of type str. The format parameters are specified according to python's [Format Specification Mini-Language](https://docs.python.org/3/library/string.html#formatspec). If an alignment character (<, >, or ^) is specified without a fixed width in the lbl_format, then the alignment will be made relative to all other <labels\>. For the str_format a missing width value will result in an alignment of all lines in the string representation of a value. The algorithm injects the width of the longest string of the <label\> strings and the width of the longest line in a <value\> string into lbl_format and str_format respectively.

The sty parameter is a string with zero or more [ANSI Select Graphic Rendition](https://en.wikipedia.org/wiki/ANSI_escape_code#SGR_(Select_Graphic_Rendition)_parameters) parameters seperated by semicolons (;).

The layout algorithm globally works as follows: 1) adjust lbl_format and str_format where necessary; 2) style and format the concatenations of each <label\> with `pointer`; 3) format the values; 4) layout each <label\>/<value\> pair in a 'pane'; 5) join these panes together with `seperator`; 6) print the panes: if `seperator` contains a newline ('\n') character then the panes are laid out from top to bottom, if not, they are laid out from left to right.

## Limitations
* Selfdocprint cannot be used in the Repl nor with `eval()`, `exec()` and `compile()`. Selfdocprint uses the `inspect` module to discover the labels and this does not work in these situations.
* Selfdocprint's print function cannot accept arguments that use the unpack operator, for example to unpack a tuple: `*my_tuple`. This does not make sense anyway because there is nothing to label the individual values of the tuple with, so it's just as informative to just use the tuple variable as is.